### 数据表示
* 状态 state: 例如 state = np.zeros((3,3),dtype=np.float32)
* 网络策略 p: 例如 p = np.array([0.1,0.2,0.05,0.1,0.4,0.15],dtype=np.float32)
* MCTS策略 pi: 同p格式一致
* 网络值 v: 为float型标量 例如 v = 0.75
* 模拟器结果 z: 为float型标量 例如 v = -1.0
* 动作 action: 为int型标量 例如 action = 2

### 超参数

* 问题规模 size: 例如 size = 19
* 输入历史表示规模 history: 例如 history = 8
* 每层卷积层核的个数 filters: 例如 filters = 256
* 残差模块的个数 n_modules: n_modules = 19
* 正则化参数 lambda_c: lambda_c = .0001
* 动作空间的大小 n_actions: n_actions = 9
* 探索与利用的平衡参数 c_puct: c_puct = 5
* 探索与利用的惯性因子 epsilon: epsilon = .25
* 探索与利用的Dirchlet分布参数 eta: eta = .03
* 退火算法参数 tau: tau = .7
* 搜索次数 n_search: n_search = 1600
* 搜索最大深度 max_search_depth: max_search_depth = 20
* 执行最大步数 max_play_steps: max_play_steps = 500
* 估值阈值 v_resign: v_resign = -.95
* 网络损失函数的迭代次数 n_iters: n_iters = 50

### 数学公式

1. annealing(arr,tau) # 退火公式，arr为计数值
2. sampling_from_dirchlet(alpha,size) # 从狄利克雷分布中采样,alpha为噪声参数，size为采样大小
3. add_noise_to_p(p,epsilon,eta) # 为策略p加入噪声eta
4. ucb(p,n_visits,c_puct) # 上置信界计算，p为策略，n_visits为访问数
5. cross_entropy(p,q) # p与q的交叉熵

### 内部函数及参数命名规范

#### Net类

1. p_op,v_op = inference(state)
2. p = get_var_value(p_op,feed_dict) 
3. v = get_var_value(v_op,feed_dict)
4. l_op = loss_op(z,v_op,pi,p_op,lambda_c) # 单个的状态的loss
5. optimizer = minimize(l_op)
6. l = get_var_value(l_op,feed_dict)
7. update() # 将 T个state，T个pi，T个z ,迭代次数n_iters传入，更新网络参数
8. 初始化state_placeholder, z_placeholder,pi_placeholder为placeholder类型
9. save_params()
10. restore_params()

#### TreeNode类
1. action, sub_node = select()
2. expand(action_priors)
3. update(leaf_value,c_puct)
4. backup(leaf_value,c_puct)

#### MCTS类
1. search() # 单次搜索
2. pi = policy(state,tao) # 多次搜索后返回pi
3. action = choice(tao) # 在当前state下给出action，是外界调用的接口
4. next_prepare() # 为下次搜索做准备，在choice中调用

#### Simulator类
1. move(action)
2. get_state(state,action)
3. render()
4. is_done() # 判断是否结束
5. who_win() # 1 代表赢， -1 代表输， 0 代表未结束 

## 流程

0. 初始化神经网络
1. 模拟一局，进行自我对弈，每一步进行n_search次MCTS模拟，
2. 一局结束后 将每一步的 s pi  z 及相应的旋转作为训练样本进行保存以供训练。
3. 显示过程和结果并保存记录
4. 训练网络，从最近的500万个样本中抽取2048个样本进行训练
5. 更新网络，每1000局后，评估当前的神经网络和以前最佳的神经网络版本（两个版本进行对弈），如果胜率达到55%以上，则使用当前的神经网络作为最佳的版本，摒弃以前的版本
6. 跳转 1. 循环 70万次
7. 效果评估

### 参考文献

tensorflow中batch normalization的用法及其L2正则化 https://www.cnblogs.com/hrlnw/p/7227447.html

AlphaGo Zero 原理 http://www.sohu.com/a/220095461_297710

#### 导入包

In [1]:
import os
import datetime

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

#### 超参数定义

In [7]:
size = 3
history = 1
state_channal = 1 # 2 * history +1
filters = 16
n_modules = 7
lambda_c = 0.0001
n_actions = size*size
c_puct = 2 
epsilon = 0.25
eta = 0.03
init_tau = 1.0
n_search = 1600
max_search_depth = 10
max_play_steps = 500
v_resign = -0.95
n_iters = 50

dir_path = "H:\\CnCqLeeds\\code\\致敬 AlphaGo Zero\\Model_save"
model_path = dir_path + "\\model_net_demo.ckpt"

#### 数学公式

In [ ]:
def annealing(arr,tau):
    """
    退火公式，arr为计数值
    """
    return np.power(arr,1.0/tau) / np.sum(np.power(arr,1.0/tau))

In [ ]:
def sampling_from_dirchlet(alpha,size):
    """
    从狄利克雷分布中采样,alpha为噪声参数，size为采样大小
    """
    eta = np.random.dirichlet([alpha]*size)
    return eta,eta[0]

In [ ]:
def add_noise_to_p(p,action,epsilon=.25,eta=0.03):
    """
    为策略p加入噪声eta
    """
    eta,_ = sampling_from_dirchlet(eta,p.size)
    res = (1-epsilon) * p + epsilon * eta
    return res,res[action]

In [ ]:
def ucb(p,n_visits,action,c_puct):
    """
    上置信界计算，p为策略，n_visits为访问数
    """
    temp = c_puct * p * np.sqrt(np.sum(n_visits)) / (1+np.array(n_visits))
    return temp, temp[action]

In [ ]:
def cross_entropy(p,q): 
    """
    p与q的交叉熵
    """
    return np.dot(np.array(p),np.log(np.array(q)))

In [ ]:
p = np.array([0.1,0.5,0.05,.2,.15])
q = np.array([.2,.3,.005,.095,.4])
n_visits = [0,0,1,0,0]


#### Net 类

In [8]:
class Net:
    def __init__(self):
        self.state_ph = tf.placeholder(tf.float32,shape=(None,size,size,state_channal))
        self.z_ph = tf.placeholder(tf.float32,shape=(None,1))
        self.pi_ph = tf.placeholder(tf.float32,shape=(None,n_actions))
        
        self.training = True # 区分训练和测试
        
        self.p_op, self.v_op = self.inference(training=self.training)
        
        self.l_op = self.loss_op()
        
        self.optimizer = self.minimize(self.training)
        
        self.saver = self.get_saver()
        
        self.sess = tf.Session()
        
        self.sess.run(tf.global_variables_initializer())
        
    def set_training(self,training=True):
        self.training = training

    def get_training(self):
        return self.training
    
    def get_saver(self):
        var_list = tf.trainable_variables()
#         print("1>>>",len(var_list))
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        var_list += bn_moving_vars
#         print("2>>>",len(var_list))
        saver = tf.train.Saver(var_list=var_list, max_to_keep=5)
        return saver
        
    def input_module(self,inputs,filters,training):
        outputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=3,padding='same')
        outputs = tf.layers.batch_normalization(outputs,training=training)
        outputs = tf.nn.relu(outputs)
        return outputs

    def residual_module(self,inputs,filters,training):
        outputs = tf.layers.conv2d(inputs,filters=filters,kernel_size=3,padding='same')
        outputs = tf.layers.batch_normalization(outputs,training=training)
        outputs = tf.nn.relu(outputs)

        outputs = tf.layers.conv2d(outputs,filters=filters,kernel_size=3,padding='same')
        outputs = tf.layers.batch_normalization(outputs,training=training)

        outputs += inputs
        outputs = tf.nn.relu(outputs)  
        return outputs
    
    def output_module(self,inputs,fc_units_v,training):
        outputs_v = tf.layers.conv2d(inputs,filters=1,kernel_size=1,padding='same')
        outputs_v = tf.layers.batch_normalization(outputs_v,training=training)
        outputs_v = tf.nn.relu(outputs_v)

        outputs_v = tf.layers.flatten(outputs_v)

        outputs_v = tf.layers.dense(outputs_v,fc_units_v,activation=tf.nn.relu)

        outputs_v = tf.layers.dense(outputs_v,1,activation=tf.nn.tanh)
        
        outputs_p = tf.layers.conv2d(inputs,filters=2,kernel_size=1,padding='same')
        outputs_p = tf.layers.batch_normalization(outputs_p,training=training)
        outputs_p = tf.nn.relu(outputs_p)
        
        outputs_p = tf.layers.flatten(outputs_p)
        
        outputs_p = tf.layers.dense(outputs_p,n_actions,activation=tf.nn.softmax)
        return outputs_p, outputs_v

    def inference(self,fc_units_v=32,num_res_module=4,training=True):
        inputs = self.input_module(self.state_ph,filters,training) # 输入模块
        
        mid_values = inputs
        for i_module in range(n_modules): # 残差模块
            mid_values = self.residual_module(mid_values,filters,training)
            
        outputs_p, outputs_v = self.output_module(mid_values,fc_units_v,training)
        return outputs_p, outputs_v
        
    def get_var_value(self,var,feed_dict):
        return self.sess.run(var,feed_dict)
    
    def loss_op(self):
        regularizer = tf.contrib.layers.l2_regularizer(scale=lambda_c)
        l2_var = tf.contrib.layers.apply_regularization(\
                    regularizer,tf.trainable_variables())
        
        l_op = l2_var + \
            tf.reduce_mean(\
                tf.reduce_sum(\
                    tf.pow(self.z_ph - self.v_op,2)\
                              - tf.matmul(self.pi_ph,tf.transpose(tf.log(self.p_op))),reduction_indices=[1]))
        return l_op
    
    def minimize(self,training=True):
        if training:
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                return tf.train.AdamOptimizer().minimize(self.l_op)
        else:
            return tf.train.AdamOptimizer().minimize(self.l_op)
    
    def update(self,feed_dict):
        for it in range(n_iters):
            self.sess.run(self.optimizer,feed_dict)
        
    def save_params(self):
        self.saver.save(self.sess,save_path=model_path)
        print(str(datetime.datetime.now())+": latest model and params saved to path:",model_path)
        
    def restore_params(self):
        self.saver.restore(self.sess,save_path=model_path)
        print(str(datetime.datetime.now())+": last model and params restored from path",model_path)
        
        

In [ ]:
net = Net()

In [ ]:
states = np.random.randn(5,3,3,1).astype(np.float32)
zs = np.ones((5,1),np.float32)
pis = np.tile(np.array([0.11]*9).astype(np.float32),(5,1))

In [ ]:
feed_dict = {net.state_ph:states,net.z_ph:zs,net.pi_ph:pis}

In [ ]:
if os.path.exists(dir_path):
    net.restore_params()
    print("Yes")
    net.update(feed_dict)
else:
    net.update(feed_dict)

print("Loss:",net.get_var_value(net.l_op,feed_dict))
net.save_params()

In [ ]:
print("Loss:",net.get_var_value(net.p_op,feed_dict))

#### TreeNode 类

In [9]:
class TreeNode:
    def __init__(self,parent,prior_p,layer):
        self._parent = parent
        self._children = dict()
        self._n_visits = 0 # 节点遍历次数
        
        self._Q = 0 # 动作平均值
        self._W = 0 # 动作累计值
        self._u = prior_p # 动作置信值, 根节点不存在置信值
        self._p = prior_p # 动作先验概率
        self.layer = layer
        
        
    def render(self):
        print(self.layer,",Q:%.3f, W:%.3f, U:%.3f, N:%3d, Prob:%.3f"%(self._Q,self._W,self._u,self._n_visits,self._p))
        for action, node in self._children.items():
            node.render()
#             if node._children:
#                 node.render()
        
        
    def select(self):
        return max(self._children.items(), key=lambda act_node: act_node[1].get_value())
    
    def get_value(self):
        return self._Q + self._u
    
    def expand(self, action_priors):
        for action, prob in action_priors:
            if action not in self._children:
                self._children[action] = TreeNode(self,prob,self.layer+1)
                
    def evaluation(self,leaf_value,c_puct):
        self._n_visits += 1
        self._W += leaf_value
        self._Q = self._W / self._n_visits
        if not self.is_root():
            self._u = c_puct * self._p * np.sqrt(self._parent._n_visits) / (1+self._n_visits)

    def is_root(self):
        return self._parent is None
    
    def is_leaf(self):
        return self._children == dict()
    
    def backup(self,leaf_value,c_puct):
        
        
        if self._parent:
            self._parent.backup(leaf_value,c_puct)
            
        self.evaluation(leaf_value,c_puct)

In [ ]:
tree = TreeNode(None,1.0,1)
tree.render()


In [ ]:
action_priors = enumerate([0.1,0.2,.3,.4])

In [ ]:
tree.expand(action_priors)
tree._children[0].expand(action_priors)
tree.render()

tree._children[2].get_value()

#### MCTS 类

In [10]:
class MCTS:
    def __init__(self,net,simulator):
        self.net = net
        self.simulator = simulator
        
        self._root = TreeNode(None,1.0,1)
        self.root = self._root
        self.c_puct = c_puct
        self.max_search_depth = max_search_depth
        self.n_search = n_search
        self.cur_depth = 0
        self.cur_node = self._root
        
        self.root_state = self.simulator.cur_state
        self.cur_state = self.root_state.copy()
        
        self.pis = []
        
        
    def search(self):
        while self.cur_depth < self.max_search_depth:
            if not self.cur_node.is_leaf():
#                 print(self.cur_state)
                
                action, self.cur_node = self.cur_node.select()
                
                self.cur_state = self.simulator.get_state(self.cur_state.copy(),action)
                
                self.cur_depth += 1
            else:
                action_priors = enumerate(self.net.get_var_value(self.net.p_op,{self.net.state_ph:self.cur_state}).reshape(-1).tolist())
                self.cur_node.expand(action_priors)
                
                leaf_value = self.net.get_var_value (self.net.v_op,{self.net.state_ph:self.cur_state})[0,0]
#                 print("leaf_value",leaf_value)
                self.cur_node.backup(leaf_value,self.c_puct)
    
                
    def policy(self,tau):
        for it in range(self.n_search):
#             print("第%d次搜索"%(it+1))
            self.search()
            self.cur_node = self._root
            self.cur_state = self.root_state
            self.cur_depth = 0
            
        pi = [x._n_visits for _,x in self._root._children.items()]
   
        # 需考虑退火 温度系数，
        pi = (np.power(pi,1/tau)/np.sum(np.power(pi,1/tau))).tolist()
        self.pis.append(pi)
        return pi
    
    def choice(self,tau):
        pi = self.policy(tau)
#         print(pi)
        action = np.random.choice(range(len(pi)),p=pi)
        self.action = action
        self.next_prepare()
        return action
    
    def next_prepare(self):
        self._root = self._root._children[self.action]
        self._root._parent = None
        self.simulator.move(self.action)
        self.root_state = self.simulator.cur_state.copy()
        self.cur_state = self.root_state.copy()
        
        self.cur_depth = 0
        self.cur_node = self._root

        return self

#### Simulator 类

In [40]:
class Simulator:
    def __init__(self):
        self.size = size
        self.cur_state = np.zeros((1,size,size,1),dtype=np.float32)
        self.records = [self.cur_state.copy()]
        self.action_records = []
        self.cur_hands = 0
        self.n_actions = size**2
        
    def move(self,action):
        self.cur_hands += 1
        state = self.cur_state
        
        if action>self.n_actions or action<0:
            print("pass")
        elif action == self.n_actions:
            print("pass")
        else:
            i = action//self.size
            j = action % self.size
            state[0,i,j,0] += 1
        self.cur_state = state
#         print("move")
        self.records.append(state.copy())
        self.action_records.append(action)
        
    def get_state(self,state,action):
        if  not (action>self.n_actions or action<0):
            i = action//self.size
            j = action % self.size
            state[0,i,j,0] += 1
            
        return state
    
    def render(self):
        print("hands:",self.cur_hands)
        print("state: \n")
        print(self.cur_state.reshape((3,3)))
        
    def is_done(self):
        if abs(np.linalg.det(state)) >= 1:
            return True
        else:
            return False
        
    def who_win(self,state):
        state = state.reshape((size,size))
        res = np.linalg.det(state)
        if res == 1.:
            return 1.
        elif res > 1.:
            return -1.
        else:
            return 0.
        
    def state_rot(self):
        pass
    def board_to_state(self):
        pass
    def state_to_board(self):
        pass

In [12]:
net = Net()


Instructions for updating:
Use the retry module or similar alternatives.


In [41]:
simulator = Simulator()

In [ ]:
simulator.render()

In [ ]:
simulator.move(3)

In [ ]:
simulator.render()

In [42]:
mcts_tree = MCTS(net,simulator)

In [ ]:
mcts_tree._root.render()

In [43]:
def get_samples(simulator,mcts_tree,z):
    feed_dict = dict()
    temp = simulator.records
    m = len(temp)
    arr = np.zeros((m,size,size,1),np.float32)

    for i in range(m):
        arr[i,:,:,]=temp[i]
        
    feed_dict['states'] = arr
    
    feed_dict['zs'] = np.tile(np.array([z]).astype(np.float32),(m,1))
    
    temp = mcts_tree.pis
    m = len(temp)
    arr = np.zeros((m,n_actions),np.float32)
    for i in range(m):
        arr[i,:]=temp[i]
        
    feed_dict['pis'] = arr
    
    return feed_dict

In [44]:
max_steps = 30

samples = dict()
for i in range(max_steps):
#     simulator.render()
    if simulator.who_win(simulator.cur_state) == 1.:
        print("Win, steps is %d"%(i+1))
        
        samples = get_samples(simulator,mcts_tree,1.)
        break
        
    else:
        action = mcts_tree.choice(1.)
#         print(action)
#         simulator.move(action)
        if i == max_steps -1 :
            samples = get_samples(simulator,mcts_tree,-1.)                        
            print("Lose, steps is %d"%(i+1))
            
# print("Records:\n",simulator.records)

Lose, steps is 30


In [45]:
samples['pis'].shape

(30, 9)

In [49]:
samples['states'][:-1].shape

(30, 3, 3, 1)

In [ ]:
feed_dict = {net.state_ph:samples['states'][:-1],net.z_ph:feed_dict['zs'],net.pi_ph:samples['pis']}

In [48]:
samples['states']

array([[[[ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.]]],


       [[[ 1.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.]]],


       [[[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.]]],


       [[[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 1.]]],


       [[[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 1.]]],


       [[[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
         [ 2.]]],


       [[[ 1.],
         [ 0.],
         [ 0.]],

        [[ 1.],
         [ 0.],
      

In [ ]:
mcts_tree.root.render()

In [ ]:
simulator.cur_state

#### Model 类

In [ ]:
class Model:
    def __init__(self):
        pass
    def play(self):
        pass
    def train(self):
        pass
    def update_net(self):
        pass
    def save_samples(self):
        pass
    def display(self):
        pass